# Homework 8
## Due Date:  Tuesday, October 31st at 11:59 PM

# Problem 1:  BST Traversal
This problem builds on Problem 1 of Homework 7 in which you wrote a binary search tree.

### Part 1

As discussed in lecture, three different types to do a depth-first traversal are: preorder, inorder, and postorder. Here is a reference: [Tree Traversal](https://en.wikipedia.org/wiki/Tree_traversal#Depth-first_search).

Write an iterator class called `DFSTraversal` with the following specifications:

* `__init__(self, tree, traversalType)`: Constructor takes a `BinaryTree` object and one of the enums from `DFSTraversalTypes`

```python
from enum import Enum

class DFSTraversalTypes(Enum):
    PREORDER = 1
    INORDER = 2
    POSTORDER = 3
```

* `changeTraversalType(self, traversalType)`: Change the traversal type
* `__iter__(self)`: This is the initialization of an iterator
* `__next__(self)`: This is called in the iterator for getting the next value

Here's how you might use your `DFSTraversal` class:

```python
input_array = [3, 9, 2, 11]
bt = BinaryTree()
for val in input_array:
    bt.insert(val)
traversal = DFSTraversal(bt, DFSTraversalTypes.INORDER)
for val in traversal:
    print(val)
2
3
9
11
```

### Part 2
Put your `BinaryTree` class (from homework 7) and your `DFSTraversal` class (from Part 1 of this homework) in a file titled `TreeTraversal.py`.

In [371]:
%%file TreeTraversal.py
class Node():
    def __init__(self,value):
        self.value = value
        self.left_child = None
        self.right_child = None
class BinaryTree():
    def __init__(self):
        self.tree = None
    def insert(self,val):
        if(self.tree == None):
            self.tree = Node(val)
        else:
            self.add(val,self.tree)
    def add(self,value,node):
        if(value < node.value):
            if(node.left_child == None):
                node.left_child = Node(value)
            else:
                self.add(value,node.left_child)
        else:
            if(node.right_child == None):
                node.right_child = Node(value)
            else:
                self.add(value,node.right_child)
    def getValues(self,depth,node=None,vals=[]):
        if(node == None):
            if(self.tree == None):
                return []
            node = self.tree 
            vals = []
        if (depth==0):
            vals.append(node.value)
        else:
            if(node.left_child != None):
                self.getValues(depth-1,node.left_child,vals)
            else:
                for i in range(int(2**(depth-1))):
                    vals.append(None)
            if(node.right_child != None):
                self.getValues(depth-1,node.right_child,vals)
            else:
                for i in range(int(2**(depth-1))):
                    vals.append(None)
        return vals

    def search(self,value, node,parent=None):
        if(node != None):
            if(value < node.value):
                return self.search(value, node.left_child,node)
            elif(value > node.value):
                return self.search(value,node.right_child,node)
            elif(node.value == value):
                return parent,node
        else:
            raise Exception()
                    
    def remove(self, val):
        parent, node = self.search(val,self.tree)
        if(parent == None):
            self.tree = None
            return
        if((node.left_child == None) and (node.right_child == None)):
            if (parent.value > val):
                parent.left_child = None
            if (parent.value < val):
                parent.right_child = None
        if((node.left_child == None) or (node.right_child == None)):
            if (node.left_child != None):
                replacing_node = node.left_child
            if (node.right_child != None):
                replacing_node = node.right_child
            if (parent.value > val):
                parent.left_child = replacing_node
            if (parent.value < val):
                parent.right_child = replacing_node
        if((node.left_child != None) and (node.right_child != None)):
            replacing_node = node.left_child
            if (parent.value > val):
                parent.left_child = replacing_node
            if (parent.value < val):
                parent.right_child = replacing_node
                
from enum import Enum

class DFSTraversalTypes(Enum):
    PREORDER = 1
    INORDER = 2
    POSTORDER = 3
    
class DFSTraversal():
    def __init__(self,tree,traversalType):
        self.tree = tree
        if(traversalType != DFSTraversalTypes.PREORDER and traversalType != DFSTraversalTypes.INORDER 
           and traversalType != DFSTraversalTypes.POSTORDER):
            raise ValueError("Must input a traversal type of the form DFSTraversalTypes.INORDER")
        else:
            self.type = traversalType
    
    def preorder(self, tree):
        if tree != None:
            yield tree.value
            yield from self.preorder(tree.left_child)
            yield from self.preorder(tree.right_child)
            
    def postorder(self, tree):
        if tree != None:
            yield from self.postorder(tree.left_child)
            yield from self.postorder(tree.right_child)
            yield tree.value
            
    def inorder(self,tree):
        if tree != None:
            yield from self.inorder(tree.left_child)
            yield tree.value
            yield from self.inorder(tree.right_child)
    
    def __iter__(self):
        if(self.type == DFSTraversalTypes.PREORDER):
            yield from self.preorder(self.tree)
        elif(self.type == DFSTraversalTypes.POSTORDER):
            yield from self.postorder(self.tree)
        else:
            yield from self.inorder(self.tree)      
    
    def changeTraversalType(self, traversalType):
        self.type = traversalType


Writing TreeTraversal.py


In [376]:
#Create a tree for test ~from question example
input_array = [3, 9, 2, 11]
bt = BinaryTree()
for val in input_array:
    bt.insert(val)

---

In [377]:
#Markov Class in Action
traversal = DFSTraversal(bt.tree, DFSTraversalTypes.INORDER)
traversal.changeTraversalType(DFSTraversalTypes.PREORDER)
for val in traversal:
    print(val)

3
2
9
11


In [378]:
traversal = DFSTraversal(bt.tree, DFSTraversalTypes.INORDER)
it = iter(traversal)
next(it)

2

## Problem 2: Markov Chains

[Markov Chains](https://en.wikipedia.org/wiki/Markov_chain) are widely used to model and predict discrete events. Underlying Markov chains are Markov processes which make the assumption that the outcome of a future event only depends on the event immediately preceeding it. In this exercise, we will be assuming that weather has Markov properties (e.g. today's weather is dependent only on yesterday's weather). We will use the Markov assumption to create a basic model for predicting weather.

To begin, let's categorize weather into 7 types: ['sunny', 'cloudy', 'rainy', 'snowy', 'windy', 'hailing'].

In the `weather.csv` file accompanying this homework, each row corresponds to one type of weather (in the order given above) and each column is the probability of one type of weather occurring the following day (also in the order given above).

The $ij$th element is the probability that the $j$th weather type occurs after the $i$th weather type. So for example, (1,2) is the probability a cloudy day occurs after a sunny day.

Take a look at the data. Make sure you see how if the previous day was sunny, the following day will have a 0.4 probability of being sunny as well. If the previous day was raining (index $i = 3$), then the following day (index $j$) has a 0.05 probability of being windy ($j = 5$).

### Part 1:  Parse the `.csv` file into a `Numpy` array

In [372]:
#Load CSV file -- hint: you can use np.genfromtxt()
import numpy as np
weather = np.genfromtxt('weather.csv', delimiter=',')

### Part 2:  Create a class called `Markov` that has the following methods:

* `load_data(array)`: loads the Numpy 2D array and stores it as a class variable.
* `get_prob(previous_day, following_day)`: returns the probability of `following_day` weather given `previous_day` weather. 

**Note:** `previous_day` and `following_day` should be passed in string form (e.g. "sunny"), as opposed to an index (e.g. 0). 




In [373]:
class Markov:
    def __init__(self):
        #array of all possible weather types
        self.weather = ['sunny', 'cloudy', 'rainy', 'snowy', 'windy', 'hailing']
        
    def load_data(self,array):
        self.data = array
    
    def get_prob(self,previous_day, following_day):
        p_index = None
        f_index = None
        for i,v in enumerate(self.weather):
            if(v == previous_day):
                p_index = i
        for i, v in enumerate(self.weather):
            if(v == following_day):
                f_index = i
        if (p_index != None and f_index != None):
            return float(self.data[p_index][f_index])
        else: 
            raise ValueError("Must input a valid weather type as a string.")
            

In [374]:
#Markov Class in Action
model = Markov()
model.load_data(weather)
model.get_prob('sunny','sunny')

0.4

---

## Problem 3: Iterators

Iterators are a convenient way to walk along your Markov chain.

#### Part 1: Using your `Markov` class from Problem 3, write `Markov` as an iterator by implementing the `__iter__()` and `__next__()` methods.

Remember:  
* `__iter__()` should return the iterator object and should be implicitly called when the loop begins
* The `__next()__` method should return the next value and is implicitly called at each step in the loop.

Each 'next' step should be stochastic (i.e. randomly selected based on the relative probabilities of the following day weather types) and should return the next day's weather as a string (e.g. "sunny") rather than an index (e.g. 0).

In [375]:
class Markov:
    def __init__(self,weather):
        #array of all possible weather types
        self.weather = ['sunny', 'cloudy', 'rainy', 'snowy', 'windy', 'hailing']
        #check that the right weather type is passed in
        if(weather not in self.weather):
            raise ValueError("Must input a valid weather type as a string.")
        else:
            self.curr = weather
        
    def load_data(self,array):
        self.data = array
    
    def get_prob(self,previous_day, following_day):
        p_index = None
        f_index = None
        for i,v in enumerate(self.weather):
            if(v == previous_day):
                p_index = i
        for i, v in enumerate(self.weather):
            if(v == following_day):
                f_index = i
        if (p_index != None and f_index != None):
            return float(self.data[p_index][f_index])
        else: 
            raise ValueError("Must input a valid weather type as a string.")
    def __iter__(self):
        return self
        
    def __next__(self):
        #get relevant weather 
        for i,v in enumerate(self.weather):
            if(v == self.curr):
                row = self.data[i]
        #create a cumulative sum of the probablities of each weather
        weather_probs = np.cumsum(row)
        #get a random number for weather
        rand_num = np.random.random()
        weather_index = weather_probs.searchsorted(rand_num)
        #return assocaite weather as string
        return self.weather[weather_index]

#### Part 2: We want to predict what weather will be like in a week for 5 different cities.

Now that we have our `Markov` iterator, we can try to predict what the weather will be like in seven days from now.

Given each city's current weather in the dictionary `city_weather` (see below), simulate what the weather will be like in 7 days from now.  Rather than just producing one prediction per city, simulate 100 such predictions per city and store the most commonly occuring prediction.

In your submission, print a dictionary `city_weather_predictions` that has each city as a key and the most commonly predicted weather as the corresponding value.

**Note**: Don't worry if your values don't seem to make intuitive sense.  We made up the weather probabilities.

In [363]:
city_weather = {
    'New York': 'rainy',
    'Chicago': 'snowy',
    'Seattle': 'rainy',
    'Boston': 'hailing',
    'Miami': 'windy',
    'Los Angeles': 'cloudy',
    'San Fransisco': 'windy'
}

In [381]:
#create dictionary
city_weather_predictions = {}

#do 100 simulations for each city weather in 7 days
for city in city_weather:
    w = city_weather[city]
    count = [0] * 6
    model = Markov(w)
    model.load_data(weather)
    for index in range(100):
        for i,v in enumerate(model):
            if (i==7):
                count[model.weather.index(v)]+=1
                break
    most_common = model.weather[count.index(max(count))]
    city_weather_predictions[city] = most_common

In [384]:
#Print city_weather_predictions dictionary
for city in city_weather_predictions:
    print(city,city_weather_predictions[city])

New York cloudy
Chicago snowy
Seattle cloudy
Boston rainy
Miami windy
Los Angeles cloudy
San Fransisco windy
